In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('../input/Kannada-MNIST/train.csv')
df_test = pd.read_csv('../input/Kannada-MNIST/test.csv')
df.head()

In [ ]:
x_train = df.iloc[:,1:].values.reshape(df.shape[0],28,28)
x_test = df_test.iloc[:,1:].values.reshape(df_test.shape[0],28,28)
y_train = df.iloc[:,0].values
id_test = df_test.iloc[:,0].values
x_train,x_val, y_train,y_val = train_test_split(x_train,y_train,stratify=y_train,test_size=0.1)

In [ ]:
x_train = x_train/255
x_val = x_val/255
x_test = x_test/255

In [ ]:
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_val = x_val.reshape(x_val.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0],28,28,1)

In [ ]:

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), strides=(1,1),padding='same',activation='relu', input_shape=(28, 28,1)),
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), strides=(2,2),padding='same',activation='relu'),
    tf.keras.layers.Dropout(0.2),
    # 2nd layer
    tf.keras.layers.Conv2D(128,kernel_size=(3,3), strides=(1,1),padding='same', activation='relu'),
    tf.keras.layers.Conv2D(128,kernel_size=(3,3), strides=(2,2),padding='same', activation='relu'),
    tf.keras.layers.Dropout(0.2),
    # 3rd layer
    tf.keras.layers.Conv2D(256,kernel_size=(3,3), strides=(1,1),padding='same', activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128, activation='relu'),    
    tf.keras.layers.Dense(10, activation='softmax')

])


In [ ]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
history = model.fit(x_train,y_train,validation_data=(x_val,y_val),epochs=1)

In [ ]:
val_loss,val_acc = model.evaluate(x_val,y_val)

In [ ]:
y_test_pred = np.argmax(model.predict(x_test),axis=1)
result = pd.DataFrame([id_test,y_test_pred],index=['id','label']).T
result = result.set_index('id')
result.to_csv('submission.csv')j